In [ ]:
import cupy as cp
import numpy as np

# Get dynamic input from user for the size of vectors
N = int(input("Enter the size of the vectors: "))

# Uncomment this block to allow dynamic input for vector values
# A = cp.array([int(input(f"Enter value for A[{i}]: ")) for i in range(N)], dtype=cp.int32)
# B = cp.array([int(input(f"Enter value for B[{i}]: ")) for i in range(N)], dtype=cp.int32)

# Comment this block if you want to provide manual input for the vectors
A = cp.random.randint(0, 10, size=N, dtype=cp.int32)
B = cp.random.randint(0, 10, size=N, dtype=cp.int32)

# Create an empty array C to store the result
C = cp.empty_like(A)

# CUDA kernel code to add vectors A and B
kernel_code = '''
extern "C" __global__
void add(const int* A, const int* B, int* C, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        C[tid] = A[tid] + B[tid];
    }
}
'''

# Compile the kernel code using CuPy's RawModule
module = cp.RawModule(code=kernel_code)
add_kernel = module.get_function("add")

# Calculate number of blocks and threads per block
threads_per_block = 256
blocks_per_grid = (N + threads_per_block - 1) // threads_per_block

# Execute the kernel to add vectors
add_kernel((blocks_per_grid,), (threads_per_block,), (A, B, C, N))

# Print the result
print("Vector A:", A.get())
print("Vector B:", B.get())
print("Addition:", C.get())


Vector A: [6 2 3 2]
Vector B: [0 6 8 4]
Addition: [ 6  8 11  6]


The goal of the code is to add two integer vectors A and B of size N, and store the result in vector C using GPU parallelism via CUDA.

⚙️ Basic Working
Memory Allocation (Host and Device):

Host memory: A, B, C are normal C++ arrays on the CPU.

Device memory: X, Y, Z are pointers allocated on the GPU using cudaMalloc.

Data Transfer (Host ⇌ Device):

The contents of A and B are copied from CPU to GPU memory using cudaMemcpy.

Kernel Launch:

The function add<<<blocks, threads>>>(...) is a CUDA kernel. It's called from the CPU but runs in parallel on the GPU.

Each thread is responsible for computing one element of the result: C[i] = A[i] + B[i].

Parallelism:

Threads are grouped into blocks, and blocks are grouped into a grid.

In your code:

cpp
Copy
Edit
int tid = blockIdx.x * blockDim.x + threadIdx.x;
This gives each thread a unique index (tid) to process one element of the vector.

Result Copy and Cleanup:

Once the GPU finishes, the result in Z is copied back to C on the CPU.

All dynamically allocated memory is freed.

📦 CUDA Concepts Used
1. Kernel Function
A special function prefixed with __global__ that runs on the GPU.

Launched from host code using triple angle brackets <<<...>>>.

2. Thread Hierarchy
threadIdx.x: Thread index within a block.

blockIdx.x: Block index within a grid.

blockDim.x: Number of threads per block.

Final thread ID = blockIdx.x * blockDim.x + threadIdx.x.

3. Memory Types
Host memory: Allocated with new or malloc.

Device memory: Allocated with cudaMalloc.

Transfers:

CPU → GPU: cudaMemcpy(..., cudaMemcpyHostToDevice)

GPU → CPU: cudaMemcpy(..., cudaMemcpyDeviceToHost)

4. Synchronization
cudaDeviceSynchronize() ensures all GPU operations are complete.

Optional but important for correctness when timing or debugging.

🚀 Why Use CUDA Here?
Parallelism: Instead of looping on CPU, we launch many threads on GPU that execute simultaneously.

Speed-up: For large arrays, this is significantly faster.

